In [22]:
import tarfile
import glob
import shutil
import SimpleITK as sitk
import numpy as np
import os
from collections import Counter
from multiprocessing import Pool
from numpy.lib.npyio import save
from numpy.lib.shape_base import _dstack_dispatcher
from numpy.lib.type_check import imag
from pathlib import Path
import pathlib
import multiprocessing
import math
import functools
from functools import partial
import time
import tqdm

In [15]:

def get_file(root_path,all_files=[]):
    '''
    递归函数，遍历该文档目录和子目录下的所有文件，获取其path
    '''
    files = os.listdir(root_path)
    for file in files:
        if not os.path.isdir(root_path + '/' + file):   # not a dir
            all_files.append(root_path + '/' + file)
        else:  # is a dir
            get_file((root_path+'/'+file),all_files)
    return all_files

# example
path = '/data/wyh/影像数据EGFR1第二部分'
all_path=get_file(path)


In [63]:
temp=list(filter(lambda x:Path(x).suffix=='.tar',all_path))#过滤掉那部分重合的文件夹，只对压缩包进行处理
temp=temp[1:]#这就是要处理的所有压缩包,第一个是已经解压过压缩包,基本上应该没有什么问题

In [62]:

def str_insert(str_origin, pos=9, str_add='_data'):
    str_list = list(str_origin)    # 字符串转list
    str_list.insert(pos, str_add)  # 在指定位置插入字符串
    str_out = ''.join(str_list)    # 空字符连接
    return  str_out

In [64]:
def remove_suffix(path):
    path = Path(path)
    return path.with_suffix("")

In [65]:
object_path=list(map(partial(str_insert,pos=9,str_add='_data'),temp))#终于掌握了偏函数的用法，会自己用了，好开心，就是重新自定义函数的一个办法
#这也是臆测的，结果就成了，他这里应该传递的是一个带参的函数
object_path=list(map(remove_suffix,object_path))

In [66]:
object_path

[PosixPath('/data/wyh_data/影像数据EGFR1第二部分/Marked/POSITIVE/3-1marked1'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/predict_PFS/1-22_1-23'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-4_6-4_NIFTI'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-10_6-10_NIFTI_6-12_6-12_NIFTI'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/4-10_4-11_4-17_4-21'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-11_6-11-0_6-11_NIFTI'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-positive/20191116positive'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-positive/2-25_2-25left_2-28_2-28left'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-positive/3-1No.1_3-1No2_3-1No3_3-1No4_3-1No5_3-5')]

In [68]:
def extract_tar(tar_path,out_path):
    for i,j in zip(tar_path,out_path):
        tf = tarfile.open(i)
        tf.extractall(i,j)


if __name__ == '__main__':
    with Pool(96) as p:
        print(p.map(partial(extract_tar,out_path=object_path),temp))



In [5]:
help(tf.extract)

Help on method extract in module tarfile:

extract(member, path='', set_attrs=True, *, numeric_owner=False) method of tarfile.TarFile instance
    Extract a member from the archive to the current working directory,
    using its full name. Its file information is extracted as accurately
    as possible. `member' may be a filename or a TarInfo object. You can
    specify a different directory using `path'. File attributes (owner,
    mtime, mode) are set unless `set_attrs' is False. If `numeric_owner`
    is True, only the numbers for user/group names are used and not
    the names.



In [3]:
tf